<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:** 

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [15]:
%%writefile /home/vertex/Desktop/pytorch-deep-learning/going_modular/going_modular/get_data.py
import os
from pathlib import Path
import requests
import zipfile
import sys

def main():
    data_path = Path("/home/vertex/Desktop/pytorch-deep-learning/going_modular/data/pizza_steak_sushi")
    if data_path.is_dir():
        print("pizza_steak_sushi dataset already exists")
        return

    print(f"Couldn't find the {data_path} directory, creating a new one and downloading the dataset...")
    data_path.mkdir(parents=True, exist_ok=True)
    
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        f.write(request.content)

    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        zip_ref.extractall(data_path)

    os.remove(data_path / "pizza_steak_sushi.zip")
    
main()

Overwriting /home/vertex/Desktop/pytorch-deep-learning/going_modular/going_modular/get_data.py


In [17]:
# Example running of get_data.py
!python /home/vertex/Desktop/pytorch-deep-learning/going_modular/going_modular/get_data.py

Couldn't find the /home/vertex/Desktop/pytorch-deep-learning/going_modular/data/pizza_steak_sushi directory, creating a new one and downloading the dataset...


In [22]:
for dirpath, dirnames, filenames in os.walk("/home/vertex/Desktop/pytorch-deep-learning/going_modular/data/pizza_steak_sushi"):
    print(f"{Path(dirpath).name} | number of directories: {len(dirnames)}, number of files: {len(filenames)}")

pizza_steak_sushi | number of directories: 2, number of files: 0
train | number of directories: 3, number of files: 0
pizza | number of directories: 0, number of files: 78
sushi | number of directories: 0, number of files: 72
steak | number of directories: 0, number of files: 75
test | number of directories: 3, number of files: 0
pizza | number of directories: 0, number of files: 25
sushi | number of directories: 0, number of files: 31
steak | number of directories: 0, number of files: 19


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular). 

In [1]:
%%writefile /home/vertex/Desktop/pytorch-deep-learning/going_modular/going_modular/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os

import torch

from torchvision import transforms

import data_setup, engine, model_builder, utils

import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--train_dir", default="/home/vertex/Desktop/pytorch-deep-learning/going_modular/data/pizza_steak_sushi/train")
parser.add_argument("--test_dir", default="/home/vertex/Desktop/pytorch-deep-learning/going_modular/data/pizza_steak_sushi/test")
parser.add_argument("--learning_rate", default=0.001, type=float)
parser.add_argument("--batch_size", default=32, type=int)
parser.add_argument("--num_epochs", default=5, type=int)
parser.add_argument("--hidden_units", default=10, type=int)

args = parser.parse_args()

# Setup hyperparameters
NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
LEARNING_RATE = args.learning_rate

# Setup directories
train_dir = args.train_dir
test_dir = args.test_dir

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")

Overwriting /home/vertex/Desktop/pytorch-deep-learning/going_modular/going_modular/train.py


In [2]:
# Example running of train.py
!python /home/vertex/Desktop/pytorch-deep-learning/going_modular/going_modular/train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

100%|█████████████████████████████████████████████| 5/5 [00:04<00:00,  1.00it/s]
[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function). 
* You may also have to write code to load in a trained model.

In [53]:
%%writefile /home/vertex/Desktop/pytorch-deep-learning/going_modular/going_modular/predict.py
import argparse
from pathlib import Path
from torchvision import transforms
import data_setup, model_builder
from PIL import Image
import torch
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
np.set_printoptions(precision=2)

matplotlib.use('TkAgg')

device = "cuda" if torch.cuda.is_available() else "cpu"
parser = argparse.ArgumentParser()
parser.add_argument("--image")

args = parser.parse_args()
img_path = Path(args.image)

data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

train_dir = "/home/vertex/Desktop/pytorch-deep-learning/going_modular/data/pizza_steak_sushi/train"
test_dir = "/home/vertex/Desktop/pytorch-deep-learning/going_modular/data/pizza_steak_sushi/test"
BATCH_SIZE = 32
HIDDEN_UNITS = 10

_, _, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

img = Image.open(img_path)

model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

model.load_state_dict(torch.load("/home/vertex/Desktop/pytorch-deep-learning/going_modular/models/05_going_modular_script_mode_tinyvgg_model.pth"))
model.eval()
with torch.inference_mode():
    out = model(data_transform(img).unsqueeze(0).to(device)).squeeze()
    pred = class_names[out.argmax()]

print(f"Predicted class: {pred}")
fig = plt.figure()
ax = fig.add_subplot(1, 2, 1)
ax.set_title("Original:")
ax.axis(False)
ax.imshow(img)

ax = fig.add_subplot(1, 2, 2)
ax.set_title("Transformed:\n"
             "(probabilities)\n"
             f"{out.softmax(0).cpu().numpy()}")
ax.axis(False)
ax.imshow(data_transform(img).permute(1, 2, 0))
fig.suptitle(f"Class: {pred}")
plt.show()

Overwriting /home/vertex/Desktop/pytorch-deep-learning/going_modular/going_modular/predict.py


In [54]:
# Example running of predict.py 
!python /home/vertex/Desktop/pytorch-deep-learning/going_modular/going_modular/predict.py --image /home/vertex/Desktop/pytorch-deep-learning/going_modular/data/pizza_steak_sushi/test/sushi/175783.jpg

/home/vertex/Desktop/pytorch-deep-learning/going_modular/going_modular/predict.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/v